In [1]:
import pandas as pd
from sqlalchemy import create_engine

## Jimmy's code below (Extract/Transform)

### EXTRACT: Store CSV into DataFrame

In [2]:
csv_file = "Resources/Data_Scientist_Salary/survey_final.csv"
data_scientist_salary_df = pd.read_csv(csv_file)
data_scientist_salary_df.head()

,version https://git-lfs.github.com/spec/v1
0,oid sha256:bcd660092021005e88b2a7051840b01a4d0...
1,size 101192916


### TRANSFORM: Create new data with select columns

In [ ]:
new_data_scientist_salary_df = data_scientist_salary_df[['field1', 'field2', 'field3']].copy()
new_data_scientist_salary_df.head()

## Sarah's code below (Extract/Transform)

### EXTRACT: Store CSV into DataFrame

In [3]:
csv_file = "Resources/HR_Analytics_DB/aug_train.csv"
hr_analytics_df = pd.read_csv(csv_file)
hr_analytics_df.head()

,enrollee_id,city,city_development_index,gender,relevent_experience,enrolled_university,education_level,major_discipline,experience,company_size,company_type,last_new_job,training_hours,target
0,8949,city_103,0.920,Male,Has relevent experience,no_enrollment,Graduate,STEM,>20,NaN,NaN,1,36,1.0
1,29725,city_40,0.776,Male,No relevent experience,no_enrollment,Graduate,STEM,15,50-99,Pvt Ltd,>4,47,0.0
2,11561,city_21,0.624,NaN,No relevent experience,Full time course,Graduate,STEM,5,NaN,NaN,never,83,0.0
3,33241,city_115,0.789,NaN,No relevent experience,NaN,Graduate,Business Degree,<1,NaN,Pvt Ltd,never,52,1.0
4,666,city_162,0.767,Male,Has relevent experience,no_enrollment,Masters,STEM,>20,50-99,Funded Startup,4,8,0.0


### TRANSFORM: Create new data with select columns

In [4]:
new_hr_analytics_df = hr_analytics_df[["enrollee_id", "education_level", "experience", "last_new_job", "target"]].copy()
new_hr_analytics_df.head()

,enrollee_id,education_level,experience,last_new_job,target
0,8949,Graduate,>20,1,1.0
1,29725,Graduate,15,>4,0.0
2,11561,Graduate,5,never,0.0
3,33241,Graduate,<1,never,1.0
4,666,Masters,>20,4,0.0


In [12]:
renamed_hr_analytics_df = new_hr_analytics_df.rename(columns = {"enrollee_id":"Candidate_ID", 
                                      "education_level":"Education_Level",
                                      "experience":"Years_Experience",
                                      "last_new_job":"Years_Tenure",
                                      "target":"Active"})
renamed_hr_analytics_df

,Candidate_ID,Education_Level,Years_Experience,Years_Tenure,Active
0,8949,Graduate,>20,1,1.0
1,29725,Graduate,15,>4,0.0
2,11561,Graduate,5,never,0.0
3,33241,Graduate,<1,never,1.0
4,666,Masters,>20,4,0.0
...,...,...,...,...,...
19153,7386,Graduate,14,1,1.0
19154,31398,Graduate,14,4,1.0
19155,24576,Graduate,>20,4,0.0
19156,5756,High School,<1,2,0.0


In [14]:
renamed_hr_analytics_df["Years_Tenure"] = renamed_hr_analytics_df["Years_Tenure"].replace({"never":"0"})

In [15]:
renamed_hr_analytics_df["Active"] = renamed_hr_analytics_df["Active"].apply(str)
renamed_hr_analytics_df["Active"] = renamed_hr_analytics_df["Active"].str.replace("0.0", "No")
renamed_hr_analytics_df["Active"] = renamed_hr_analytics_df["Active"].str.replace("1.0", "Yes")

In [16]:
renamed_hr_analytics_df.dtypes

Candidate_ID         int64
Education_Level     object
Years_Experience    object
Years_Tenure        object
Active              object
dtype: object

In [17]:
renamed_hr_analytics_df.count()

Candidate_ID        19158
Education_Level     18698
Years_Experience    19093
Years_Tenure        18735
Active              19158
dtype: int64

In [18]:
dropped_hr_analytics_df = renamed_hr_analytics_df.dropna(how= "any")
dropped_hr_analytics_df

,Candidate_ID,Education_Level,Years_Experience,Years_Tenure,Active
0,8949,Graduate,>20,1,Yes
1,29725,Graduate,15,>4,No
2,11561,Graduate,5,0,No
3,33241,Graduate,<1,0,Yes
4,666,Masters,>20,4,No
...,...,...,...,...,...
19153,7386,Graduate,14,1,Yes
19154,31398,Graduate,14,4,Yes
19155,24576,Graduate,>20,4,No
19156,5756,High School,<1,2,No


In [19]:
Final_hr_analytics_df = dropped_hr_analytics_df.sort_values(by = "Candidate_ID")
Final_hr_analytics_df

,Candidate_ID,Education_Level,Years_Experience,Years_Tenure,Active
17321,1,High School,2,0,No
6730,2,Phd,15,3,Yes
10024,4,Masters,9,>4,No
15817,5,Masters,10,1,No
4552,7,Masters,13,1,No
...,...,...,...,...,...
17935,33375,Graduate,16,1,No
5009,33376,Masters,8,4,No
3351,33377,Graduate,10,2,No
4756,33379,Graduate,18,2,No


## Luis's code below (Load)

### Connect to postgres database: data_scientist_DB

In [ ]:
from config import key 
rds_connection_string = f"postgres:{key}@localhost:5432/data_scientist_DB"
engine = create_engine(f'postgresql://{rds_connection_string}')

### Check for tables already created

In [ ]:
# Tables have already been created with create_table_statements.sql
engine.table_names()

### Combine dataframes into a single one

In [ ]:
combined_df = pd.merge(new_data_scientist_salary_df, hr_analytics_df, on="field1") 

### LOAD: Use pandas to load combined DataFrame into database

In [ ]:
combined_df.to_sql(name='data_scientist_tbl', con=engine, if_exists='append', index=False)

### Confirm data has been added by querying the customer_name table
* NOTE: can also check using pgAdmin

In [ ]:
pd.read_sql_query('select * from data_scientist_tbl', con=engine).head()